### Segmentation with yolov8

이전에 Mask R-CNN을 활용하여 segmentation task를 수행했었다. 이번에는 yolov8에서 제공하는 모델로 segmentation을 수행한다.

### Dataset Format
우선, Yolov8의 input으로 사용될 데이터를 준비해야 한다. yolov8의 경우, 원본 이미지와 라벨 정보를 담은 txt파일이 필요하다. 해당 txt파일의 형식은 아래와 같다.

```
<class-index> <x1> <y1> <x2> <y2> ... <xn> <yn>
```
위의 x좌표와 y좌표의 경우, 모두 이미지의 가로 세로 크기에 대해 정규화된 형태이다.

반면에, AI Hub에서 방출한 데이터셋의 경우 아래와 같은 구조를 가지고 있다.

```
(... 생략 ...)
"X Coordinate": [210, 209, 213, 222 ... ], 
"Y Coordinate": [1927, 1915, 1905, 1897 ... ], 
(... 생략 ...)
```

정규화도 진행되어 있지 않을 뿐더러 json 파일 형태로 구성되어 있기 때문에 요구하는 데이터 셋 형식에 맞게 변환한다.

### 분할하고자 하는 클래스
미국 선녀 벌레, 썩덩 나무 노린재, 흰점 도둑 나방의 3개 클래스이다.

In [3]:
import os
import cv2
import json
import shutil

In [5]:
# class number를 이름을 기준으로 정의
class_num = {  
    "미국선녀벌레" : 0,
    "썩덩나무노린재" : 1,
    "흰점도둑나방" : 2
    }

classes_dir = "./Data/labels/"
class_names = os.listdir(classes_dir)

origin_image_dir = "./Data/images"  # 원본 이미지가 담겨있는 dir

image_saved_dir = "./yolo_data/images"  # 이미지를 복사하여 저장할 경로
label_saved_dir = "./yolo_data/labels"  # yolo label 형식에 맞춰 저장할 경로

for class_ in class_names:  # 클래스 파일들을 순회하면서
    label_dir = os.path.join(classes_dir, class_)  # json file dir (ex. ./Data/labels/class1)
    image_dir = os.path.join(origin_image_dir, class_)  # image file dir (ex. ./Data/images/class1)
    label_filenames = os.listdir(label_dir)  # json filenames
    for filename in label_filenames:
        json_path = os.path.join(label_dir, filename)  # json file full path (ex. ./Data/labels/class1/~.json)

        with open(json_path, "r", encoding="utf8") as file:  # json file open with full path
            json_file = json.load(file)

        speices_name = json_file["Meta"]["Scientific_Name"]
        if speices_name != "Bradysia impatiens":
            image_filename = json_file["Meta"]["Image_FileName"]  # image filename
            image_path = os.path.join(image_dir, image_filename)  # image file full path
            image_extension = image_filename[image_filename.rfind(".") : ]  # image file's extension
            
            image = cv2.imread(image_path)  # read image
            height, width, _ = image.shape  # get height, width from image for normalization

            annotation_txt = ""  # segmenation 좌표 정보를 담을 빈 문자열
            annotations = json_file["Image_Annotation"]["Annotations"]  # annotation 정보를 가져오고
            for annotation in annotations:  # 각 annotation 정보들을 순회하면서
                instance_name = annotation["Object Name"]  # 병해충 이름을 가져오고
                coordinates = f"{class_num[instance_name]}"  # 클래스 번호를 담은 문자열을 생성한 뒤

                # x, y좌표들을 정규화해서
                normed_x_coordinate = [i / width for i in annotation["X Coordinate"]] 
                normed_y_coordinate = [i / height for i in annotation["Y Coordinate"]]

                for x_coordinate, y_coordinate in zip(normed_x_coordinate, normed_y_coordinate):
                    append_txt = f" {x_coordinate} {y_coordinate}"  # x y 순서로 문자열을 생성해서
                    coordinates += append_txt  # 별도의 문자열과 합친 뒤
                annotation_txt += coordinates + "\n"  # 빈 문자열에 한 개의 annotation에 대한 정보를 합친다

            with open(os.path.join(label_saved_dir, image_filename.replace(image_extension, ".txt")), "w", encoding="utf8") as label:
                label.write(annotation_txt)

            label.close()
            shutil.copyfile(image_path, os.path.join(image_saved_dir, image_filename))
            file.close()


### Split Dataset
데이터의 형식을 맞춰 재구성하였다. 이제 Train/Validation/Test 셋을 6 : 2 : 2 비율로 분할하자.

In [6]:
import splitfolders

splitfolders.ratio("./yolo_data", "./bug_dataset", seed=11, ratio=(0.6, 0.2, 0.2))

Copying files: 0 files [00:00, ? files/s]

Copying files: 2330 files [03:24, 11.38 files/s]


### Model Training
모델을 학습시킨다.

In [ ]:
from ultralytics import YOLO

# 사용한 이미지 장 수 : 약 1000장, 소요된 시간 : 3시간 11분
model = YOLO('yolov8l-seg.yaml').load('yolov8l.pt')
results = model.train(
    data='./bug_segmentation.yaml', 
    epochs=100, 
    imgsz=640,
    batch=16,
    save=True,
    device=0,
    workers=2,
    translate=0.0
    )

In [9]:
custom_model = YOLO(r"C:\Users\admin\Desktop\CudaTest\runs\segment\train3\weights\best.pt")
results = custom_model.predict(source="./bug_dataset/test/images/*", save=True)


image 1/233 d:\CudaTest\segmentation\bug_dataset\test\images\20220808_113039__718.png: 640x576 1 Metcalfa_pruinosa, 62.1ms
image 2/233 d:\CudaTest\segmentation\bug_dataset\test\images\20220808_120039__135.png: 640x576 2 Metcalfa_pruinosas, 46.8ms
image 3/233 d:\CudaTest\segmentation\bug_dataset\test\images\20220808_130039__377.png: 640x576 2 Metcalfa_pruinosas, 30.8ms
image 4/233 d:\CudaTest\segmentation\bug_dataset\test\images\20220808_143039__366.png: 640x576 2 Metcalfa_pruinosas, 43.0ms
image 5/233 d:\CudaTest\segmentation\bug_dataset\test\images\20220808_143039__653.png: 640x576 1 Metcalfa_pruinosa, 38.8ms
image 6/233 d:\CudaTest\segmentation\bug_dataset\test\images\20220808_150039__638.png: 640x576 2 Metcalfa_pruinosas, 48.2ms
image 7/233 d:\CudaTest\segmentation\bug_dataset\test\images\20220808_150039__737.png: 640x576 2 Metcalfa_pruinosas, 37.5ms
image 8/233 d:\CudaTest\segmentation\bug_dataset\test\images\20220808_153039__398.png: 640x576 2 Metcalfa_pruinosas, 60.5ms
image 9/2

### Auto Annotation Test(With ultralytics' SAM)

ultralytics에서 제공하는 SAM(Segment Anything Model)의 경우 1,100만개의 엄선된 이미지에 10억 개 이상의 polygon이 포함된 방대한 데이터셋으로 학습되어 모델의 이름 말 그대로 어떠한 물체든 polygon을 생성해준다.

### Auto Annotation
해당 모델을 기반으로 오토 어노테이션을 진행할 때는 annotation 박스 좌표를 인수로 입력하는지에 대한 여부에 따라 결과가 달라진다. 어노테이션 박스 좌표를 제공하면 해당 어노테이션 박스 안의 객체를 자동으로 segmentation을 진행하여 생성된 mask를 기반으로 txt파일을 제공하는 형태로 진행되고, 아무런 인수도 전달하지 않으면 사전학습된 SAM 모델 내부에 학습된 모든 객체에 대해 segmentation을 진행하고 txt파일을 반환한다.

만약 본인이 mask를 생성하고자 하는 대상이 특수한 대상이라면, 사전에 훈련된 segmentation model과 object detection 모델을 기반으로 auto annotation을 진행할 수 있다.

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
from ultralytics.data.annotator import auto_annotate

auto_annotate(
    data="./auto_annotation_input", 
    det_model=r"D:\CudaTest\cabbage\detection_model\cabbage_detection8\weights\best.pt", 
    sam_model="sam_b.pt", 
    output_dir="./auto_annotation_test"
    )


image 1/18 d:\CudaTest\segmentation\auto_annotation_input\V006_79_1_05_03_03_11_1_1299b_20201023_23.JPG: 480x640 1 black_rot, 207.0ms
image 2/18 d:\CudaTest\segmentation\auto_annotation_input\V006_79_1_05_03_03_11_1_1299b_20201023_23_a0003.JPG: 480x640 1 black_rot, 21.6ms
image 3/18 d:\CudaTest\segmentation\auto_annotation_input\V006_79_1_05_03_03_11_1_1299b_20201023_23_a0004.JPG: 480x640 1 black_rot, 7.0ms
image 4/18 d:\CudaTest\segmentation\auto_annotation_input\V006_79_1_05_03_03_11_1_1299b_20201023_23_a0005.JPG: 640x480 1 black_rot, 199.3ms
image 5/18 d:\CudaTest\segmentation\auto_annotation_input\V006_79_1_05_03_03_11_1_1299b_20201023_23_a0006.JPG: 640x480 1 black_rot, 18.4ms
image 6/18 d:\CudaTest\segmentation\auto_annotation_input\V006_79_1_05_03_03_11_1_1299b_20201023_23_a0007.JPG: 480x640 1 black_rot, 15.5ms
image 7/18 d:\CudaTest\segmentation\auto_annotation_input\V006_79_1_05_03_03_11_1_1299b_20201023_23_a0008.JPG: 480x640 1 black_rot, 28.2ms
image 8/18 d:\CudaTest\segmenta

In [35]:
import cv2
import numpy as np

image = cv2.imread("./auto_annotation_input/V006_79_1_05_03_03_11_1_4902r_20200916_2_a0004.jpg")
height, width, _ = image.shape

with open("./auto_annotation_test/V006_79_1_05_03_03_11_1_4902r_20200916_2_a0004.txt", "r", encoding="utf8") as txt:
    lines = txt.read()
points = [float(i) for i in lines[2:].split(" ")]
points_x = [int(points[i]*width) for i in range(0, len(points), 2)]
points_y = [int(points[i]*height) for i in range(1, len(points), 2)]

point = list(zip(points_x, points_y))
point_list = np.array([list(i) for i in point], np.int32)

txt.close()

img = cv2.polylines(image, [point_list], isClosed=True, color=(0, 255, 0), thickness=4)
cv2.imwrite("./test.png", img)
cv2.imshow("polylines", img)
cv2.waitKey(0)
cv2.destroyAllWindows()